# Equazioni differenziali
## Studio delle traiettorie di spostamento di un gruppo di studenti lungo un corridoio
<u> Simulazione dell'uscita di scuola</u>

In [11]:
import pandas as pd
import numpy  as np
import math

### Configurazione dei parametri della simulazione
#### Ricordiamo le equazioni del moto
\begin{cases}
    X(t)={\tau} \times V_{0} \times\cos({\theta}) \times(1-\exp(-\frac{t}{\tau}))      & \quad \text{con } {\tau} \text{ parametro caratteristico del moto smorzato.}
\\& \quad \text{ Il valore di riferimento è stato scelto considerando che per t = 5 }{\tau}
\\& \quad \text{ la funzione è prossima al valore asintotico}
    \\
    Y(t)=  V_{0} \times \sin({\theta}) \times t
    \\
\end{cases}
#### Nella simulazione sono stati adottati i seguenti parametri:
\begin{cases}
    \tau  & \quad \text{con } {\tau} \text{ Variabile aleatoria con distribuzione uniforme } U(\frac{1}{2},\frac{3}{2}) \times tau_{mean}\\
    V_{0}   & \quad \text{con }V_{0} \text{ Variabile aleatoria con distribuzione uniforme } U(\frac{1}{2},\frac{3}{2})\times v_{mean}\\
    \theta & \quad \text{con } {\theta} \text{ Variabile aleatoria con distribuzione uniforme } U(\frac{16}{40},\frac{19}{40})\times \pi \\
\end{cases}


In [ ]:
tau_mean = 1
v_mean   = 1

#### Configurazione della geografia della scuola e della popolazione studentesca
Sono state effettuate 3 simulazioni per le quali riportiamo di seguito i parametri di configurazione:

##### Simulazione modello Pre-COVID
\begin{cases}
    sezioni = ['A','B','C','D']  & \quad \text{Sono state considerate le sezioni dalla "A" alla "D"} \\
    corso   = [1,2,3,4,5]  & \quad \text{Sono stati considerati 5 anni di corso per ogni sezione} \\
    s \tilde{=} N(\mu,\sigma)  & \quad \text{Gli studenti presenti nella singola classe sono stati modellati come una v.a. Normale con media }\mu=20 \text{ e varianza } \sigma =5\\
    l_{corridoio} = 6 [m] & \quad \text{Larghezza del corridoio in metri} \\
    l_{aula} = 5 [m] & \quad \text{Larghezza dell'aula in metri} \\
\end{cases}

##### Simulazione modello Levi Civita
\begin{cases}
    sezioni = ['A','B']  & \quad \text{Sono state considerate metà delle sezioni per simulare l'alternanza settimanale} \\
    corso   = [1,2,3,4,5]  & \quad \text{Sono stati considerati 5 anni di corso per ogni sezione} \\
    s \tilde{=} N(\mu,\sigma)  & \quad \text{Gli studenti presenti nella singola classe sono stati modellati come una v.a. Normale con media }\mu=20 \text{ e varianza } \sigma =5\\
    l_{corridoio} = 6 [m] & \quad \text{Larghezza del corridoio in metri} \\
    l_{aula} = 5 [m] & \quad \text{Larghezza dell'aula in metri} \\
\end{cases}
##### Simulazione modello D'Assisi
\begin{cases}
    sezioni = ['A','B','C','D']  & \quad \text{Sono state considerate le sezioni dalla "A" alla "D"} \\
\\
    corso   = [1,2,3,4,5]  & \quad \text{Sono stati considerati 5 anni di corso per ogni sezione} \\
    s \tilde{=} N(\mu,\sigma)  & \quad \text{Gli studenti presenti nella singola classe sono stati modellati come una v.a. Normale con media }\mu=10 \text{ e varianza } \sigma =2.5 \\ & \quad \text{ dimezzate rispetto al caso Pre-Covid per tenere conto dell'attività in DAD per metà degli studenti}\\
    l_{corridoio} = 6 [m] & \quad \text{Larghezza del corridoio in metri} \\
    l_{aula} = 5 [m] & \quad \text{Larghezza dell'aula in metri} \\
\end{cases}


In [12]:
file='scenario_01_all'
p_mean, p_sigma = 20, 5
sezioni = ['A','B','C','D']

#file='scenario_02_lcivita'
#p_mean, p_sigma = 20, 5
#sezioni = ['A','B']

#file='scenario_03_dassisi'
#p_mean, p_sigma = 10, 2.5
#sezioni = ['A','B','C','D']

p_mean, p_sigma = 20, 5

sezioni = ['A','B','C','D']
corso   = [1,2,3,4,5]

l       = len(sezioni)*len(corso)
pos     = {}
l_aula  = 5
#larghezza (dx) e lunghezza (dy) del corridoio
dx,dy   = 6,l_aula*(int(l/2)+1)
for i in range(int(l/2)+1):
    pos[2*i]={2*i:[l_aula,l_aula*i+2.5]}
    pos[2*i+1]={2*i+1:[l_aula+dx,l_aula*i+2.5]}

#Popolo le classi col numero di alunni
s       = np.random.normal(p_mean, p_sigma, l)
s       = s.astype('int')

#Associo gli alunni alle aule
aule    = {}
id      = 0
mtr     = 0
xsz,ysz = 0,0
for sz in sezioni:
    for c in corso:
        for i in range(s[id]):
            mtr = mtr+1
            aule[mtr]={'sezione':sz
            ,'anno':c
            ,'matricola':'MTR'+('00000'+str(mtr))[-5:]
            ,'x':pos[id][id][0]
            ,'y':pos[id][id][1]
            ,'tau':np.random.uniform(tau_mean/2,3*tau_mean/2)
            ,'V0':np.random.uniform(v_mean/2,3*v_mean/2)
}
        id = id+1

df_aule = pd.DataFrame.from_dict(aule,orient='index')
df_aule = df_aule.reset_index(inplace=False)


### Incapsuliamo la descrizione del moto lungo l'asse Y in una funzione
\begin{cases}
    Y(t)=  V_{0} \times \sin({\theta}) \times t \\
    s_{0}   & \quad \text{Posizione iniziale lungo l'asse Y } \\
    a_{0}   & \quad \text{Componente lungo Y dell'accelerazione al tempo } t_{0}\\
    V_{0}   & \quad \text{Componente lungo Y della velocità al tempo } t_{0}\\
    T       & \quad \text{Periodo di riferimento entro cui valutare il moto ed espresso in secondi} \\
\end{cases}

In [13]:
# Y axes model mouvement
def sy(s0,a0,v0,t0,T):
    '''
    spazio
    '''
    step=[]
    #a0=max(0,a0)
    #v0=max(0,v0)
    for t in range(0,T):
        if t<t0:
            step.extend([s0])
        else:
            ss=0.5*a0*(t-t0)*(t-t0)+v0*(t-t0)+s0
            step.extend([ss])
    return step

### Incapsuliamo la descrizione del moto lungo l'asse X in una funzione
\begin{cases}
    X(t)={\tau} \times V_{0} \times\cos({\theta}) \times(1-\exp(-\frac{t}{\tau})) \\
    t_{0}   & \quad \text{Istante in cui uno studente esce dall'aula dopo lo squillo della campanella }\\
    s_{0}   & \quad \text{Posizione iniziale lungo l'asse X } \\
    X_{min} & \quad \text{Limite inferiore del corridoio lungo X } \\
    X_{max} & \quad \text{Limite superiore del corridoio lungo X } \\
    V_{0}   & \quad \text{Componente lungo X della velocità al tempo } t_{0}\\
    {\tau}  & \quad \text{Parametro di riferimento per il moto smorzato }\\
    T       & \quad \text{Periodo di riferimento entro cui valutare il moto ed espresso in secondi} \\
\end{cases}


In [14]:
# X axes model mouvement
def sx(s0,xmin,xmax,tau,v0,t0,T):
    '''
    spazio
    '''
    step=[]
    xmax=xmax-0.5
    xmin=xmin+0.5
    for t in range(0,T):
        if t<t0:
            step.extend([s0])
        else:
            ss=s0+ v0*(t-t0)+v0*tau*(1-math.exp(-(t-t0)/tau))
            if ss>xmax:
                ss=xmax
            if ss<xmin:
                ss=xmin
            step.extend([ss])
    return step

In [15]:
T=5*60
S = pd.DataFrame()
for i in df_aule.index:
    #print(i)
    t0  = np.random.randint(60)
    teta= np.random.uniform(16*math.pi/40,19*math.pi/40)
    if df_aule.iloc[i]['x']>dx:
        p=pd.DataFrame([sx(s0=df_aule.iloc[i]['x']
            ,xmin=l_aula,xmax=l_aula+dx
            ,tau=df_aule.iloc[i]['tau'],v0=-df_aule.iloc[i]['V0']*math.cos(teta)
            ,t0=t0,T=T)
        ,sy(s0=df_aule.iloc[i]['y']
        ,a0=0,v0=df_aule.iloc[i]['V0']*math.sin(teta)
        ,t0=t0,T=T)]).T
    else:
        p=pd.DataFrame([sx(s0=df_aule.iloc[i]['x']
            ,xmin=l_aula,xmax=l_aula+dx
            ,tau=df_aule.iloc[i]['tau'],v0=df_aule.iloc[i]['V0']*math.cos(teta)
            ,t0=t0,T=T)
        ,sy(s0=df_aule.iloc[i]['y']
        ,a0=0,v0=df_aule.iloc[i]['V0']*math.sin(teta)
        ,t0=t0,T=T)]).T
    p.columns=['x','y']
    p['sezione']    = df_aule.iloc[i]['sezione']
    p['anno']       = df_aule.iloc[i]['anno']
    p['matricola']  = df_aule.iloc[i]['matricola']
    p['tau']         = df_aule.iloc[i]['tau']
    p['V0']         = df_aule.iloc[i]['V0']
    S=S.append(p)


In [16]:
S.to_csv(file+'.dat',sep=';')

In [17]:
S.index.name='t'
S=S.reset_index()

In [18]:
stat=S[S.x>l_aula]
stat=stat[stat.x<l_aula+dx]
stat=stat[stat.y<=60]
stat['S']=np.int32(stat['y'].values)
#stat.index.name='t'

In [19]:
stat=stat.groupby(by=['t','S']).count()

In [20]:
stat.to_csv(file+'.stat',';')